<a href="https://colab.research.google.com/github/rileighdethy/mgmt467-analytics-portfolio/blob/main/Lab1_AI_Assisted_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1: AI‑Assisted SQL Foundations (Colab)**
**Course:** MGMT 467 — AI‑Assisted Big Data Analytics in the Cloud  
**When:** Unit 1 · Week 3 (Thursday)  
**Goal:** Use *Gemini as a co‑pilot* to write and understand basic SQL against the **Superstore** dataset in **BigQuery**.  
**You will practice:** Prompt engineering → SQL generation → Query execution → Interpretation.

> ✅ **Deliverable (submit link in Brightspace):** A completed `Lab1_AI_Assisted_SQL.ipynb` with prompts, code, outputs.


## ✅ What you need before starting
- A Google account with access to **Google Colab**.
- A **Google Cloud Project** with **BigQuery** API enabled.
- A dataset named `superstore_data` and a table named `sales` (your instructor or TA will provide details).
- (Optional) A GitHub repo to store your work for your team.


## 0) Install and import libraries

In [2]:
!pip -q install google-cloud-bigquery google-cloud-bigquery-storage db-dtypes pandas pyarrow kagglehub

In [3]:
from google.cloud import bigquery
from google.colab import auth
import pandas as pd
import os, json, textwrap, pathlib, pprint
print("Libraries imported.")

Libraries imported.


## 1) Authenticate to GCP and set your project
Run the cell below. When prompted, authorize Colab to access your Google Cloud resources.

In [4]:
auth.authenticate_user()
print("✅ Authenticated.")

# Set your GCP project (EDIT THIS!)
PROJECT_ID = 'mgmt-467-471613'   # <-- REPLACE 'mgmt-467-471613' with your actual GCP project id.
# assert PROJECT_ID != "mgmt-467-471613", "Please set PROJECT_ID to your GCP project id."
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# BigQuery client
bq = bigquery.Client(project=PROJECT_ID)
print("✅ BigQuery client created for:", PROJECT_ID)

✅ Authenticated.
✅ BigQuery client created for: mgmt-467-471613


### Quick connection test (optional)
This will try to read the first 5 rows from your Superstore table. If it fails, double‑check your dataset/table name.

In [5]:
DATASET = "superstore_data"
TABLE   = "sales"
BQ_TABLE = f"`{PROJECT_ID}.{DATASET}.{TABLE}`"

query = f"SELECT * FROM {BQ_TABLE} LIMIT 5"
try:
    df_preview = bq.query(query).to_dataframe()
    display(df_preview)
    print("✅ Connection OK.")
except Exception as e:
    print("⚠️ Could not query the table. Error below:")
    print(e)

,Category,City,Country,Customer_ID,Customer_Name,Discount,Market,_________,Order_Date,Order_ID,...,Sales,Segment,Ship_Date,Ship_Mode,Shipping_Cost,State,Sub_Category,Year,Market2,weeknum
0,Office Supplies,Los Angeles,United States,LS-172304,Lycoris Saunders,0,US,1,00:00.0,CA-2011-130813,...,19,Consumer,00:00.0,Second Class,4.37,California,Paper,2011,North America,2
1,Office Supplies,Los Angeles,United States,MV-174854,Mark Van Huff,0,US,1,00:00.0,CA-2011-148614,...,19,Consumer,00:00.0,Standard Class,0.94,California,Paper,2011,North America,4
2,Office Supplies,Los Angeles,United States,CS-121304,Chad Sievert,0,US,1,00:00.0,CA-2011-118962,...,21,Consumer,00:00.0,Standard Class,1.81,California,Paper,2011,North America,32
3,Office Supplies,Los Angeles,United States,CS-121304,Chad Sievert,0,US,1,00:00.0,CA-2011-118962,...,111,Consumer,00:00.0,Standard Class,4.59,California,Paper,2011,North America,32
4,Office Supplies,Los Angeles,United States,AP-109154,Arthur Prichep,0,US,1,00:00.0,CA-2011-146969,...,6,Consumer,00:00.0,Standard Class,1.32,California,Paper,2011,North America,40


✅ Connection OK.


## 2) (Optional) Download a dataset with **KaggleHub**
Use **KaggleHub** to pull sample data locally into Colab (e.g., for offline exploration or to stage for GCS).
> This is **optional** for Lab 1 (we focus on BigQuery), but useful to practice data access workflows.
> https://www.kaggle.com/datasets/anandaramg/global-superstore

In [6]:
import kagglehub, os, pathlib

DATASET_REF = "anandaramg/global-superstore"   # you can change this to any public dataset reference

download_path = kagglehub.dataset_download(DATASET_REF)
print("Path to dataset files:", download_path)

# List files
p = pathlib.Path(download_path)
print("Files:")
for f in p.glob("**/*"):
    if f.is_file():
        print("-", f)


100%|██████████| 3.18M/3.18M [00:00<00:00, 135MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/anandaramg/global-superstore/versions/1
Files:
- /root/.cache/kagglehub/datasets/anandaramg/global-superstore/versions/1/Global Superstore.txt


## 3) Prompting approach for Lab 1
You'll **paste prompts into Gemini** (in a separate tab), get back **SQL**, then paste that SQL here to execute.  
We’ll practice three core question types:
- **“What”** (lists & filters) → `SELECT`, `WHERE`, `DISTINCT`
- **“How many”** (counts by category) → `COUNT`, `GROUP BY`
- **“Who is best”** (rank/limit by metric) → `SUM`, `ORDER BY`, `LIMIT`

### 3A) “What” Question — `SELECT`, `WHERE`, `DISTINCT`
**Business Question:** *A manager wants a list of all **unique** product sub‑categories sold in the **West** region.*

**Paste this prompt into Gemini (no edits needed except the project id if you used a different table path):**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Find all the unique values in the 'Sub_Category' column, but only for rows where the 'Region' column is exactly 'West'.
```

**Then paste the SQL you get into the cell below** (replace the placeholder) and run it.

In [7]:
sql_str = """
-- PASTE the SQL from Gemini here
SELECT DISTINCT Sub_Category
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
WHERE Region = 'West'
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_what = bq.query(sql_str).to_dataframe()
print(f"Rows: {len(df_what)}")
display(df_what.head(20))

Rows: 17


,Sub_Category
0,Paper
1,Art
2,Storage
3,Appliances
4,Supplies
5,Envelopes
6,Fasteners
7,Labels
8,Furnishings
9,Accessories


**Explain it back (metacognition):** In Gemini, paste:  
> *Explain the following SQL query line by line: [paste your SQL]*  
Summarize what you learned here:

> **Notes:** the keyword DISTINCT ensures that only unique values are returned, so you won't see duplicate categories. FROM specifies the table where you're querying from. WHERE Region = "West" satisfies the condition to only pull from where region is exactly west

### 3B) “How many” Question — `COUNT`, `GROUP BY`
**Business Question:** *How many orders were placed in each **Ship Mode**?*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Count the total number of records for each unique value in the 'Ship_Mode' column.
# The final result should have two columns: 'Ship_Mode' and 'order_count'.
```


In [8]:
sql_str = """
-- PASTE the SQL from Gemini here
SELECT Ship_Mode, COUNT(*) AS order_count
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
GROUP BY Ship_Mode
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_howmany = bq.query(sql_str).to_dataframe()
display(df_howmany)

,Ship_Mode,order_count
0,Second Class,10309
1,Standard Class,30775
2,Same Day,2701
3,First Class,7505


### 3C) “Who is best” Question — `SUM`, `ORDER BY`, `LIMIT`
**Business Question:** *Identify the **top 5 most profitable customers**.*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`. The customer identifier is 'Customer_ID'.
# GOAL: Calculate the sum of 'Profit' for each customer. The final output should show the 'Customer_ID' and their total profit, sorted from highest to lowest profit, and limited to only the top 5 results.
```


In [9]:
sql_str = """
-- PASTE the SQL from Gemini here
SELECT Customer_ID, SUM(Profit) AS total_profit
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
GROUP BY Customer_ID
ORDER BY total_profit DESC
LIMIT 5
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_best = bq.query(sql_str).to_dataframe()
display(df_best)

,Customer_ID,total_profit
0,TC-209804,8981.3239
1,RB-193604,6976.0959
2,SC-200954,5757.4119
3,HL-150404,5622.4292
4,AB-101054,5444.8055


## 4) Challenge prompts (author your own)
Write **your own precise prompt** in a text cell (or comment) for each question below, then get SQL from Gemini and run it in the provided code cells.

**Challenge 1:** *What is the **average discount** for products in the **Technology** category sold in the **East** region?*  
**Challenge 2:** *How many **unique customers** has each **Segment** (Consumer/Corporate/Home Office) served?*

**Gemini Prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is '[YOUR_PROJECT_ID].superstore_data.sales'.
# GOAL: Calculate the average value of the 'Discount' column for rows where the 'Category' column is 'Technology' and the 'Region' column is 'East'.
```

In [10]:
sql_str = """
-- PASTE your Gemini-generated SQL here
SELECT AVG(Discount) AS avg_discount
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
WHERE Category = 'Technology' AND Region = 'East'
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)
df_ch1 = bq.query(sql_str).to_dataframe()
display(df_ch1)

,avg_discount
0,0.0


**Gemini Prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is '[YOUR_PROJECT_ID].superstore_data.sales'.
# GOAL: Count the number of unique customers for each 'Segment'.
```

In [12]:
sql_str = """
-- PASTE your Gemini-generated SQL here
SELECT Segment, COUNT(DISTINCT Customer_ID) AS unique_customers
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
GROUP BY Segment
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)
df_ch2 = bq.query(sql_str).to_dataframe()
display(df_ch2)

,Segment,unique_customers
0,Consumer,2509
1,Home Office,907
2,Corporate,1457


## 5) Reflection (DIVE mindset)
- **Discover:** What did you find first?  
I thought that Gemini did a really good job with the prompts that it was given. They were clear and easy to understand with what was being asked.
- **Investigate:** What alternate query or filter changed the story?  
DISTINCT, AVG, or COUNT are really important queries and hold a lot of information in what you're trying to find.
- **Validate:** Where could the AI‑generated SQL be wrong or incomplete? How did you check?  
I have experience with SQL, so I always double checked to make sure what it was asking was clear enough for gemini to understand and that it's output was correct.
- **Extend:** Which stakeholder could use your results tomorrow? What action should they take?
I think this could be applicable in a lot of instances, it would just need to be determined what information you're trying to find and if the data contains what's necessary to find it.

## 6) Save your work to GitHub (pick one of the options)
**Option A (recommended):** In Colab, go to **File → Save a copy in GitHub…** and select your team repo + folder (e.g., `labs/Unit1/`).  
**Option B (CLI, if you know git):**
```bash
# (In Colab) mount Drive, then clone/pull/push as usual with a PAT
# Be careful to NOT store secrets in the notebook.
```
Name the file **`Lab1_AI_Assisted_SQL.ipynb`** and push it to your team repo.